# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import cifar10
import os

TensorFlow 2.x selected.


In [3]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [4]:
print ( trainX.shape)
print ( testX.shape)
print ( trainY.shape)
print ( testY.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [5]:
Y_0_4 = np.where(trainY <= 4) 
X_0_4 = trainX[Y_0_4[0]]
X_0_4.shape 

(25000, 32, 32, 3)

In [6]:
Y_5_9 = np.where(trainY >= 5)
X_5_9= trainX[Y_5_9[0]] 
X_5_9.shape

(25000, 32, 32, 3)

In [7]:
Ytest_0_4 = np.where(testY <= 4)
Xtest_0_4= testX[Ytest_0_4[0]] 
Xtest_0_4.shape 

(5000, 32, 32, 3)

In [8]:
Ytest_5_9 = np.where(testY >= 5)
Xtest_5_9 = testX[Ytest_5_9[0]] 
Xtest_5_9.shape

(5000, 32, 32, 3)

In [9]:
train_0_4y = trainY[Y_0_4]
test_0_4y= testY[Ytest_0_4]
test_0_4y.shape

(5000,)

In [10]:
train_5_9y = trainY[Y_5_9]
test_5_9y= testY[Ytest_5_9]
test_5_9y.shape

(5000,)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [11]:
Y_04 = tf.keras.utils.to_categorical(train_0_4y)
testY04 = tf.keras.utils.to_categorical(test_0_4y)
Y_59 = tf.keras.utils.to_categorical(train_5_9y,num_classes=10)
testY59 = tf.keras.utils.to_categorical(test_5_9y,num_classes =10)
Y_04.shape


(25000, 5)

In [12]:
Y59 = Y_59[:,5:10] 
print(Y59.shape)
test_Y_59 = testY59[:,5:10] 
print(test_Y_59.shape)

(25000, 5)
(5000, 5)


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout,Activation
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import Accuracy

In [0]:
tf.keras.backend.clear_session()
# Sequential
model = Sequential()

# 1st CNN layer
model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_0_4.shape[1:]))
model.add(Activation('relu'))

# Max pool layer with 20% dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20)) 

# 2nd CNN layer 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

# Batcth normalizing
model.add(BatchNormalization())

# 3rd CNN layer 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

# 4th CNN layer 
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

# max pool  2 with 30% dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.30)) 

# Flattening the layer 
model.add(Flatten())

# Dens layer 
model.add(Dense(512,activation ='relu'))

# Output layer 
model.add(Dense(5,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        1

In [0]:
mckpt = tf.keras.callbacks.ModelCheckpoint('./cifar.h5', monitor='val_acc', save_best_only=True,)

In [17]:
history = model.fit(X_0_4,Y_04,          
          validation_data=(Xtest_0_4,testY04),
          epochs=10,
          batch_size=32, callbacks=[mckpt],workers=2)

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 85s 3ms/sample - loss: 1.0361 - accuracy: 0.5764 - val_loss: 0.8536 - val_accuracy: 0.6526
Epoch 2/10
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.8295 - accuracy: 0.6713 - val_loss: 0.7487 - val_accuracy: 0.7074
Epoch 3/10
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.7325 - accuracy: 0.7114 - val_loss: 0.7424 - val_accuracy: 0.7134
Epoch 4/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.6592 - accuracy: 0.7484 - val_loss: 0.7303 - val_accuracy: 0.7172
Epoch 5/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.6104 - accuracy: 0.7673 - val_loss: 0.5988 - val_accuracy: 0.7652
Epoch 6/10
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.5560 - accuracy: 0.7889 - val_loss: 0.6137 - val_accuracy: 0.7686
Epoch 7/10
25000/25000 [==============================] -

In [18]:
history = model.fit(X_0_4,Y_04,          
          validation_data=(Xtest_0_4,testY04),
          epochs=15,
          initial_epoch=10,
          batch_size=32, callbacks=[mckpt],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 11/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.3640 - accuracy: 0.8624 - val_loss: 0.5867 - val_accuracy: 0.7906
Epoch 12/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.3339 - accuracy: 0.8766 - val_loss: 0.5858 - val_accuracy: 0.8022
Epoch 13/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.3220 - accuracy: 0.8797 - val_loss: 0.6902 - val_accuracy: 0.7700
Epoch 14/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.2985 - accuracy: 0.8902 - val_loss: 0.6178 - val_accuracy: 0.7800
Epoch 15/15
25000/25000 [==============================] - 84s 3ms/sample - loss: 0.2735 - accuracy: 0.9001 - val_loss: 0.6044 - val_accuracy: 0.8042


In [0]:
#Model accuracy achived as 80

In [0]:
model.save_weights('weight.hdf5')

In [0]:
model.save('model.hdf5') 

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [0]:
model_59 =  load_model('model.hdf5')
for layer in model_59.layers[:13]:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

In [23]:
model_59.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_59.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 32)        1

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
mckpt59 = tf.keras.callbacks.ModelCheckpoint('./cifar59.h5', 
                                           monitor='val_acc', save_best_only=True,)

In [25]:
history59 = model_59.fit(X_5_9,Y59,          
          validation_data=(Xtest_5_9,test_Y_59),
          epochs=20,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.9375 - accuracy: 0.6932 - val_loss: 0.6554 - val_accuracy: 0.7620
Epoch 2/20
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.5240 - accuracy: 0.8089 - val_loss: 0.5477 - val_accuracy: 0.8084
Epoch 3/20
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.4490 - accuracy: 0.8365 - val_loss: 0.5792 - val_accuracy: 0.7994
Epoch 4/20
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.3924 - accuracy: 0.8567 - val_loss: 0.5786 - val_accuracy: 0.8000
Epoch 5/20
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.3430 - accuracy: 0.8745 - val_loss: 0.6283 - val_accuracy: 0.7968
Epoch 6/20
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.2952 - accuracy: 0.8910 - val_loss: 0.6127 - val_accuracy: 0.8096
Epoch 7/20
25000/25000 [==============================] -

In [28]:
history59 = model_59.fit(X_5_9,Y59,          
          validation_data=(Xtest_5_9,test_Y_59),
          epochs=25,
          initial_epoch=20,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 21/25
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.0991 - accuracy: 0.9648 - val_loss: 0.8962 - val_accuracy: 0.8150
Epoch 22/25
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.0970 - accuracy: 0.9661 - val_loss: 0.8553 - val_accuracy: 0.8204
Epoch 23/25
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.0980 - accuracy: 0.9666 - val_loss: 0.8340 - val_accuracy: 0.8136
Epoch 24/25
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.0881 - accuracy: 0.9687 - val_loss: 0.8668 - val_accuracy: 0.8148
Epoch 25/25
25000/25000 [==============================] - 32s 1ms/sample - loss: 0.0864 - accuracy: 0.9702 - val_loss: 0.8702 - val_accuracy: 0.8198


In [53]:
history59 = model_59.fit(X_5_9,Y59,          
          validation_data=(Xtest_5_9,test_Y_59),
          epochs=30,
          initial_epoch=25,
          batch_size=32, callbacks=[mckpt59],workers=4)

Train on 25000 samples, validate on 5000 samples
Epoch 26/30
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.0891 - accuracy: 0.9698 - val_loss: 0.8515 - val_accuracy: 0.8206
Epoch 27/30
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.0881 - accuracy: 0.9697 - val_loss: 0.9557 - val_accuracy: 0.8068
Epoch 28/30
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.0805 - accuracy: 0.9726 - val_loss: 0.9692 - val_accuracy: 0.8068
Epoch 29/30
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.0803 - accuracy: 0.9728 - val_loss: 0.9867 - val_accuracy: 0.8130
Epoch 30/30
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.0775 - accuracy: 0.9734 - val_loss: 0.9056 - val_accuracy: 0.8176


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [31]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [34]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [35]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [36]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline

In [0]:
# instantiate the vectorizer
vector = TfidfVectorizer()

In [0]:
vector = TfidfVectorizer(stop_words='english')
dtm_train = vector.fit_transform(twenty_train.data)
dtm_test = vector.transform(twenty_test.data)
features = vector.get_feature_names()

In [49]:
print(vector.get_feature_names()[-10:])

['zvi', 'zvonko', 'zwart', 'zyeh', 'zyklon', 'zyxel', 'zz', 'zzz', 'ªl', 'íålittin']


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

log_Regression = LogisticRegression()

In [51]:
log_Regression.fit(dtm_train, twenty_train.target)
y_pred_class = log_Regression.predict(dtm_test)
print('test Accuracy ',metrics.accuracy_score(twenty_test.target, y_pred_class))
y_pred_class_train = log_Regression.predict(dtm_train)
print('train accuracy ',metrics.accuracy_score(twenty_train.target, y_pred_class_train))

test Accuracy  0.9054593874833555
train accuracy  0.9968985378821444


In [52]:
# Classification report  
print(metrics.classification_report(twenty_test.target, y_pred_class))

              precision    recall  f1-score   support

           0       0.96      0.76      0.85       319
           1       0.87      0.98      0.92       389
           2       0.94      0.90      0.92       396
           3       0.88      0.95      0.91       398

    accuracy                           0.91      1502
   macro avg       0.91      0.90      0.90      1502
weighted avg       0.91      0.91      0.90      1502

